In [48]:
from transformers import pipeline
import torch
import pandas as pd
from PIL import Image
import os
import evaluate
import math


torch.cuda.is_available()

True

In [62]:
# Prepare input samples

real_cat_dir = "archive/CAT_04"
tom_dir = "archive/tom_and_jerry/tom_and_jerry/tom"
real_dog_dir = "archive/n02110063-malamute"
cartoon_dog_dir = "archive/cartoon_dogs_val_images"

img_folder = cartoon_dog_dir
images = []
files = []

for file in os.listdir(img_folder):
    try:
        img_path = os.path.join(img_folder, file)
        img = Image.open(img_path)
        images.append(img)
        files.append(file)
    except IOError:
        print(f"Could not read image: {file}")

In [65]:
# Prepare baseline models
classifier = pipeline(model="efederici/convnext-base-224-22k-1k-orig-cats-vs-dogs",  device=0)

results = classifier(images)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [66]:
evaluation_results = []
sum_of_confidence = 0
for file, result in zip(files, results):
    print(f"Results for {file}: {result}")
    evaluation_results.append(int(result[0]["label"] == "Dog"))
    sum_of_confidence += result[0]["score"]
print(f"Average confidence: {sum_of_confidence/len(evaluation_results)}")

Results for 1076.jpg: [{'label': 'Dog', 'score': 0.9983548521995544}, {'label': 'Cat', 'score': 0.0016450758557766676}]
Results for 809.jpg: [{'label': 'Dog', 'score': 0.973152756690979}, {'label': 'Cat', 'score': 0.02684727869927883}]
Results for 737.jpg: [{'label': 'Dog', 'score': 0.9996005892753601}, {'label': 'Cat', 'score': 0.0003994413127657026}]
Results for 1239.jpg: [{'label': 'Dog', 'score': 0.892386794090271}, {'label': 'Cat', 'score': 0.10761323571205139}]
Results for 1277.jpg: [{'label': 'Dog', 'score': 0.9954407215118408}, {'label': 'Cat', 'score': 0.004559307359158993}]
Results for 713.jpg: [{'label': 'Dog', 'score': 0.8238659501075745}, {'label': 'Cat', 'score': 0.17613404989242554}]
Results for 889.jpg: [{'label': 'Dog', 'score': 0.6903964877128601}, {'label': 'Cat', 'score': 0.3096035420894623}]
Results for 1058.jpg: [{'label': 'Dog', 'score': 0.9911337494850159}, {'label': 'Cat', 'score': 0.00886627472937107}]
Results for 1146.jpg: [{'label': 'Dog', 'score': 0.9992052

In [42]:
# Calculate the accuracy for cats and dogs
def calculate_accuracy(results):
    clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
    for ref, pred in zip(results, [1 for _ in range(len(results))]):
        clf_metrics.add(references=ref, predictions=pred)
    return clf_metrics.compute()   

In [67]:
calculate_accuracy(evaluation_results)

{'accuracy': 0.890625,
 'f1': 0.9421487603305785,
 'precision': 0.890625,
 'recall': 1.0}

In [27]:
# Prepare cluster models
# imagenet-1k
classifier = pipeline(model="JAW1453/swin-tiny-patch4-window7-224-DDD2", device=0)
classifier(test_img)

classifier = pipeline(model="nicolasdupuisroy/vit-gabor-detection-v2", device=0)
classifier(test_img)

[{'label': 'absent', 'score': 0.5122044682502747},
 {'label': 'present', 'score': 0.48779550194740295}]

In [28]:
# imagenet-21k
classifier = pipeline(model="alirzb/S1_M1_R1_beit_42507336", device=0)
classifier(test_img)

/opt/conda/lib/python3.11/site-packages/transformers/models/beit/feature_extraction_beit.py:28: FutureWarning: The class BeitFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use BeitImageProcessor instead.
  warnings.warn(


[{'label': 'None_seizures', 'score': 0.5324092507362366},
 {'label': 'Seizures', 'score': 0.46759071946144104}]